In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/cleaned_reviews.tsv", sep="\t")

In [3]:
df.head()

,rating,vote,verified,reviewTime,reviewerName,reviewText,summary
0,5,0,True,"07 14, 2014",Alvey,great read every issue,Five Stars
1,3,0,True,"06 24, 2014",Jeff A Milne,magazine great time technology magazine new st...,Great While it lasted
2,5,0,True,"06 1, 2014",P. R THOMPSON,ive read maximum pc mpc many year article tech...,For Both the Newbie and The Advanced PC User
3,5,0,True,"05 15, 2014",Earle Williams,order magazine grandson then 7 go 30 wasis dep...,Great Gift
4,4,0,True,"03 28, 2014",JJM,subscribed nook version year thoroughly enjoy ...,Decent magazine at a good price
